# Analysis of Embeddings

## Loading Weakly Labeled data

In [ ]:
import pandas as pd

df = pd.read_parquet('../data/weak_labelled/log_reg_weak_labeling_weaklabels.parquet')
df.head(5)

df['embedding_vec'].loc[0].shape

## Visualizing Labeled data
To project the high dimensional embeddings into a humanly readable format we implemented Arize's Phoenix app that allows us to interactively look at the embedding space projected down into 3 dimensions by UMAP.

In [ ]:
import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

from src.px_utils import create_dataset, launch_px

test_ds = create_dataset("Weak Labeled Dataset", df, df['embedding_vec'], df['label'])

launch_px(test_ds, None)

`Cluster 0` seems to clearly stand out from the other clusters. This cluster consists almost solely of music album reviews that were labeled with `1`, meaning a positive sentiment.

Otherwise, the weak labeled embeddings don't show a specific pattern in the UMAP projection made by Phoenix.